In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import DataLoader, Dataset
from regressor import CNNModel  # Ensure this imports the updated model definition
from loader import load, augment, resize

class CustomDataset(Dataset):
    def __init__(self, images, points, transform=None):
        self.images = images
        self.points = points
        self.transform = transform

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        points = self.points[idx]

        if self.transform:
            image, points = self.transform(image, points)

        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        points = torch.tensor(points, dtype=torch.float32)
        return image, points

def train_model(data_dir, epochs=50, batch_size=32, model_path='model.pth'):
    # Load dataset
    print(f"Loading data from directory: {data_dir}")
    images, points = load(data_dir)
    print(f"Loaded {len(images)} images and {len(points)} points.")
    
    if len(images) == 0 or len(points) == 0:
        print("No data found. Exiting.")
        return

    dataset = CustomDataset(images, points)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = CNNModel()
    
    # Check if GPU is available and move model to GPU if it is
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, points in train_loader:
            images, points = images.to(device), points.to(device)  # Move tensors to GPU
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, points)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, points in val_loader:
                images, points = images.to(device), points.to(device)  # Move tensors to GPU
                outputs = model(images)
                loss = criterion(outputs, points)
                val_loss += loss.item() * images.size(0)
        
        val_loss = val_loss / len(val_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Save the best model
    torch.save(model.state_dict(), model_path)

if __name__ == '__main__':
    data_dir = '/home/navid/Desktop/VisionProject/four-corners'
    train_model(data_dir)
